In [1]:
from sentence_transformers import SentenceTransformer
from llama_index import VectorStoreIndex, SimpleDirectoryReader, get_response_synthesizer
from llama_index import Document as llama_index_doc
import chromadb
from chromadb.utils import embedding_functions
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import django
import os; os.chdir(r"D:/Projects/RAG-webapp")
from main.utilities.utils_testRetreival import create_rag, add_docs, index_builder
from main.utilities.helper_functions import create_folder, get_first_words, copy_folder_contents, hash_file
from typing import Union, List, Dict, Any

D:\Projects\venv2\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
binary_path: D:\Projects\venv2\lib\site-packages\bitsandbytes\cuda_setup\libbitsandbytes_cuda116.dll
CUDA SETUP: Loading binary D:\Projects\venv2\lib\site-packages\bitsandbytes\cuda_setup\libbitsandbytes_cuda116.dll...


D:\Projects\venv2\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
CUDA extension not installed.
CUDA extension not installed.
D:\Projects\venv2\lib\site-packages\transformers\modeling_utils.py:4664: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
Some weights of the model checkpoint at TheBloke/Mistral-7B-Instruct-v0.2-GPTQ were not used when initializing MistralForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model

In [2]:
tests_path = "tests"
collection_name = "vdb_test1"

collection_path = os.path.join(tests_path, f'collection_{collection_name}')
docs_path = os.path.join(collection_path, "docs")

In [3]:
# Embedding function
model_name = "paraphrase-multilingual-MiniLM-L12-v2"
embed_model = SentenceTransformer(model_name)

In [4]:
create_rag(collection_path)
create_folder(docs_path)
index = index_builder(collection_path)

# Connect to Chroma vector DB
client = chromadb.PersistentClient(path=collection_path)
collection = client.get_collection("default")

In [5]:
def index_docs(collection_path: str, docs_paths: Union[List[str] ,str]):

    if isinstance(docs_paths, str):
        docs_paths = [docs_paths]

    index = index_builder(collection_path)

    for doc_path in docs_paths:
        doc_name = os.path.splitext(os.path.basename(doc_path))[0]
        document = SimpleDirectoryReader(input_files=[doc_path]).load_data()

        for idx, chunked_doc in enumerate(document):
            doc = llama_index_doc(text=chunked_doc.text, id_=f"{doc_name}_{idx}")
            index.insert(doc)

Index documents to vdb

In [6]:
docs_paths = [
    r"D:\Projects_tmp\RAG_docs\Modern_World_History_ch9_Industrialism.pdf",
    r"D:\Projects_tmp\RAG_docs\Modern_World_History_ch7_FrenchRevolution.pdf",
    r"D:\Projects_tmp\RAG_docs\Modern_World_History_ch12_Global_Transformations.pdf",
    r"D:\Projects_tmp\RAG_docs\Modern_World_History_ch16_WW2.pdf",
    ]

index_docs(collection_path, docs_paths)


D:\Projects\venv2\lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Test retrieval

In [45]:
def retrieve_relevant_docs(query: str, collection_path: str, top_k: int = 10, print_results: bool = True):
    client = chromadb.PersistentClient(path=collection_path)
    collection = client.get_collection("default")

    query_emb = embed_model.encode(query).reshape(1, -1)

    # Get all documents from vector store
    results = collection.get(include=['embeddings', 'documents'])

    chunk_texts = results['documents']
    chunk_embs = np.array(results['embeddings'])

    # Cosine similarity
    similarities = cosine_similarity(query_emb, chunk_embs)[0]

    sorted_indices = np.argsort(similarities)[::-1][:top_k]

    retrieved_contexts = []
    for idx in sorted_indices:
        retrieved_contexts.append((round(similarities[idx], 4), chunk_texts[idx]))
    
    if print_results:
        print("\nTop-k similar chunks:\n")
        for idx in sorted_indices:
            print(f"Similarity: {similarities[idx]:.4f}")
            print(chunk_texts[idx] + "...\n")
            print("----------------------------------")

    return similarities, retrieved_contexts

In [46]:
query = "Mexico, France, invasion, Napoleon" 
query = "The French Invade Mexico The end of the civil war did not bring an end to Mexico’s troubles, though. Exiled conservatives plotted with some Europeans to reconquer Mexico."
# query = "هجوم مکزیک فرانسه ناپلئون" 
query_emb = embed_model.encode(query).reshape(1, -1)

res = retrieve_relevant_docs(query, collection_path, top_k = 15, print_results = True)



Top-k similar chunks:

Similarity: 0.7602
by launching a rebellion against the liberal government in 1858. They enjoyed some
early successes in battle and seized control of Mexico City. The liberals kept up the
fight from their headquarters in the city of Veracruz. Eventually the liberals gained
the upper hand and, after three years of bitter civil war, they defeated the rebels.Juárez became president of the reunited country after his election in 1861.
The French Invade Mexico The end of the civil war did not bring an end to
Mexico’ s troubles, though. Exiled conservatives plotted with some Europeans toreconquer Mexico. In 1862, French ruler Napoleon III responded by sending alarge army to Mexico. Within 18 months, France had taken over the country.Napoleon appointed Austrian Archduke Maximilian to rule Mexico as emperor.Juárez and other Mexicans fought against French rule. After five years under siege,the French decided that the struggle was too costly. In 1867, Napoleon ordered thea

In [48]:
similarities = res[0]

In [50]:
# results.keys() = dict_keys(['ids', 'embeddings', 'metadatas', 'documents'])

similarities.shape

(662,)

In [9]:
relevant_chunk = """by launching a rebellion against the liberal government in 1858. They enjoyed some
early successes in battle and seized control of Mexico City. The liberals kept up the
fight from their headquarters in the city of Veracruz. Eventually the liberals gained
the upper hand and, after three years of bitter civil war, they defeated the rebels.
Juárez became president of the reunited country after his election in 1861.
The French Invade Mexico The end of the civil war did not bring an end to
Mexico’s troubles, though. Exiled conservatives plotted with some Europeans to
reconquer Mexico. In 1862, French ruler Napoleon III responded by sending a
large army to Mexico. Within 18 months, France had taken over the country.
Napoleon appointed Austrian Archduke Maximilian to rule Mexico as emperor.
Juárez and other Mexicans fought against French rule. After five years under siege,
the French decided that the struggle was too costly. In 1867, Napoleon ordered the
army to withdraw from Mexico. Maximilian was captured and executed.
Juárez was reelected president of Mexico in 1867. He returned to the reforms he
had proposed more than ten years earlier. He began rebuilding the country, which had
been shattered during years of war. He promoted trade with
foreign countries, the opening of new roads, the building of
railroads, and the establishment of a telegraph service. He set
up a national education system separate from that run by the
Catholic Church. In 1872, Juárez died of a heart attack. But
after half a century of civil strife and chaos, he left his country a legacy of relative peace, progress, and reform."""
relevant_emb = embed_model.encode(relevant_chunk).reshape(1, -1)

similarity = cosine_similarity(query_emb, relevant_emb)[0][0]
print(f"\nSimilarity between query and expected chunk: {similarity:.4f}")



Similarity between query and expected chunk: 0.5246


In [10]:
query_list = [
    "What is ATP and how is it produced?",
    "Explain the function of the nucleus in a cell.",
    "What is the process of transcription in biology?"
]
relevant_chunks = [
    "ATP is produced in mitochondria through oxidative phosphorylation...",
    "The nucleus stores genetic material...",
    "Transcription is the process where DNA is converted into RNA..."
]

correct = 0
for query, ground in zip(query_list, relevant_chunks):
    q_emb = embed_model.encode(query).reshape(1, -1)
    ground_emb = embed_model.encode(ground).reshape(1, -1)
    sim = cosine_similarity(q_emb, ground_emb)[0][0]
    print(f"\nQuery: {query}\nSimilarity: {sim:.4f}")
    if sim > 0.5:
        correct += 1

recall = correct / len(query_list)
print(f"\nRecall @1: {recall:.2f}")


Query: What is ATP and how is it produced?
Similarity: 0.6535

Query: Explain the function of the nucleus in a cell.
Similarity: 0.7065

Query: What is the process of transcription in biology?
Similarity: 0.8429

Recall @1: 1.00
